In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
loader = PyPDFLoader(file_path="./tailwind.pdf")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [ ]:
len(docs)

In [ ]:
embeddings = OpenAIEmbeddings()
index_name = "faiss_index_tailwind"
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local(index_name)

In [ ]:
# embeddings = OpenAIEmbeddings()
new_vectorstore = FAISS.load_local("faiss_index_tailwind", embeddings)

In [ ]:
memory = ConversationBufferWindowMemory(k=2)
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    ),
    memory=memory,
    chain_type="stuff",
    retriever=new_vectorstore.as_retriever(),
)

In [ ]:
qa.run("Who are the authors of the tailwind book?")

In [ ]:
qa.run("How can I use the typography plugin?")

In [ ]:
qa.run("Give me an example to create a responsive layout")

In [ ]:
qa.run("How can I create a menu bar that's always on top using tailwind")

In [ ]:
qa.run("How can I customize tailwind")

In [ ]:
qa.run("What kind of customization can I do?")

In [ ]:
memory.load_memory_variables({})